In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv

dotenv.load_dotenv()

/Users/jiaronghe/Desktop/projects/react-native/coffee-appetit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY") 
openai_model_name = os.getenv("OPENAI_EMBEDDING_MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [ ]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

### Try out embeddings

In [ ]:
output = client.embeddings.create(input = ["helloo there"],model=model_name)
embedings = output.data[0].embedding

In [ ]:
print(embedings)

In [ ]:
len(embedings)

### Wrangle dataset

In [ ]:
df=pd.read_json('products/products.jsonl',lines=True)

In [ ]:
df.head(2)

In [ ]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [ ]:
df['text'].head()

In [ ]:
texts = df['text'].tolist()

In [ ]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [ ]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

### Generate Embeddings

In [ ]:
output = client.embeddings.create(input = texts,model=model_name)

In [ ]:
embeddings = output.data

### Push data to database

In [ ]:
index_name = "coffee-appetit"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

### Get Closest documents

In [ ]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [ ]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)